In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
import logging
from pathlib import Path

In [2]:
def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        force=True
    )

def analyze_electrode_characteristics(hup_atlas_path: str):
    """
    Analyze electrode characteristics regarding resection and SOZ
    
    Args:
        hup_atlas_path: Path to HUP atlas .mat file
    """
    logger = logging.getLogger(__name__)
    
    # Load HUP atlas
    logger.info("Loading HUP atlas...")
    hup_atlas = sio.loadmat(hup_atlas_path)
    
    # Get electrode information
    resected = hup_atlas['resected_ch'].flatten().astype(bool)
    soz = hup_atlas['soz_ch'].flatten().astype(bool)
    
    # Calculate statistics
    total_electrodes = len(resected)
    resected_count = np.sum(resected)
    soz_count = np.sum(soz)
    both_count = np.sum(resected & soz)
    good_electrodes = np.sum(~(resected | soz))
    
    # Calculate percentages
    resected_pct = (resected_count / total_electrodes) * 100
    soz_pct = (soz_count / total_electrodes) * 100
    both_pct = (both_count / total_electrodes) * 100
    good_pct = (good_electrodes / total_electrodes) * 100
    
    # Print results
    logger.info("\nElectrode Analysis Results:")
    logger.info(f"Total electrodes: {total_electrodes}")
    logger.info(f"\nCounts:")
    logger.info(f"Resected electrodes: {resected_count} ({resected_pct:.1f}%)")
    logger.info(f"SOZ electrodes: {soz_count} ({soz_pct:.1f}%)")
    logger.info(f"Both resected and SOZ: {both_count} ({both_pct:.1f}%)")
    logger.info(f"Good electrodes (neither): {good_electrodes} ({good_pct:.1f}%)")
    
    # Analyze overlap
    only_resected = np.sum(resected & ~soz)
    only_soz = np.sum(~resected & soz)
    
    logger.info("\nDetailed Breakdown:")
    logger.info(f"Only resected: {only_resected}")
    logger.info(f"Only SOZ: {only_soz}")
    logger.info(f"Both: {both_count}")
    logger.info(f"Neither: {good_electrodes}")
    
    # Create patient-level statistics
    patient_nos = hup_atlas['patient_no'].flatten()
    unique_patients = np.unique(patient_nos)
    
    logger.info("\nPatient-level Statistics:")
    logger.info(f"Total patients: {len(unique_patients)}")
    
    patient_stats = []
    for pat in unique_patients:
        pat_mask = patient_nos == pat
        pat_stats = {
            'patient_id': pat,
            'total_electrodes': np.sum(pat_mask),
            'resected': np.sum(resected[pat_mask]),
            'soz': np.sum(soz[pat_mask]),
            'both': np.sum((resected & soz)[pat_mask]),
            'good': np.sum(~(resected | soz)[pat_mask])
        }
        patient_stats.append(pat_stats)
    
    patient_df = pd.DataFrame(patient_stats)
    logger.info("\nPer-patient Summary Statistics:")
    logger.info(f"Average electrodes per patient: {patient_df['total_electrodes'].mean():.1f}")
    logger.info(f"Average resected electrodes per patient: {patient_df['resected'].mean():.1f}")
    logger.info(f"Average SOZ electrodes per patient: {patient_df['soz'].mean():.1f}")
    logger.info(f"Average good electrodes per patient: {patient_df['good'].mean():.1f}")
    
    return {
        'total_electrodes': total_electrodes,
        'resected_mask': resected,
        'soz_mask': soz,
        'good_mask': ~(resected | soz),
        'patient_stats': patient_df
    }

def main():
    setup_logging()
    logger = logging.getLogger(__name__)
    
    try:
        DATA_DIR = Path('../Data')
        hup_atlas_path = DATA_DIR / 'HUP_atlas.mat'
        
        # Analyze electrodes
        stats = analyze_electrode_characteristics(hup_atlas_path)
        
        # Save results
        RESULTS_DIR = Path('../results')
        RESULTS_DIR.mkdir(exist_ok=True)
        
        # Save patient statistics
        stats['patient_stats'].to_csv(RESULTS_DIR / 'electrode_patient_stats.csv', index=False)
        
        # Save boolean masks
        np.save(RESULTS_DIR / 'electrode_masks.npy', {
            'resected': stats['resected_mask'],
            'soz': stats['soz_mask'],
            'good': stats['good_mask']
        })
        
        logger.info("\nResults saved to results directory")
        
    except Exception as e:
        logger.error(f"Error during analysis: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2024-11-21 19:21:09,184 - __main__ - INFO - Loading HUP atlas...
2024-11-21 19:21:09,187 - __main__ - ERROR - Error during analysis: Reader needs file name or open file-like object


OSError: Reader needs file name or open file-like object